In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from point2mn import point2mn
from point2mn import get_main_points
torch.manual_seed(1)

In [2]:
import numpy as np
import pandas as pd

In [3]:
embedding_index = {}

f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

print('found word vecs: ',len(embedding_index))

found word vecs:  400000


In [4]:
# sorted([s for s in  list(embedding_index.keys()) if s[0].isalpha()])

In [5]:
main_points = pd.read_csv('DebateStar/Meta Data/metadata_appended_main_points.csv') 
main_points.dropna(subset = ['For_Main_Points', 'against_Main_Points'], inplace = True)
main_points = main_points.reset_index(drop = True)
main_points.head()

,id,title,date,for,against,For_Main_Points,against_Main_Points
0,d20191112,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...
1,d20191029,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']","[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia..."
2,d20191022,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....
3,d20190917,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",['The United States government should follow t...,['Individuals should have the freedom to choos...
4,d20190802,The Recent U.S. Policy Towards China Is Produc...,2019-08-02,"['Michael Pillsbury', 'Kori Schake']","['Graham Allison', 'Jake Sullivan']",['The prospect of China becoming an open and l...,['The United States and China are great compet...


In [6]:
results = pd.read_csv('DebateStar/results_data/final_online.csv') 
results = results.loc[results['winner'].apply(lambda x: x != 'undecided')]
results['winner'] = results['winner'].apply(lambda x: 1 if x == 'for' else 0)
id2winner = results[['id', 'winner']].set_index('id').to_dict()['winner']

In [7]:
main_points['label'] = main_points.id.apply(lambda x: id2winner[x] if x in id2winner else np.nan)
main_points = main_points.dropna(subset = ['label']).reset_index(drop = True)
main_points.head()

,id,title,date,for,against,For_Main_Points,against_Main_Points,label
0,d20191112,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...,0.0
1,d20191029,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']","[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia...",1.0
2,d20191022,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....,0.0
3,d20190917,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",['The United States government should follow t...,['Individuals should have the freedom to choos...,0.0
4,d20190802,The Recent U.S. Policy Towards China Is Produc...,2019-08-02,"['Michael Pillsbury', 'Kori Schake']","['Graham Allison', 'Jake Sullivan']",['The prospect of China becoming an open and l...,['The United States and China are great compet...,0.0


In [8]:
inputs = main_points.id.tolist()
labels =  main_points.label.tolist()

In [9]:
from sklearn.model_selection import train_test_split

In [76]:
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size = 0.2, random_state = 5)

In [77]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 0)

In [78]:
y_train = torch.tensor(y_train)
y_val = torch.tensor(y_val)
y_test = torch.tensor(y_test)


In [79]:
pd.Series(y_test).value_counts(normalize = True)

0.0    0.5
1.0    0.5
dtype: float64

In [80]:
pd.Series(y_val).value_counts(normalize = True)

1.0    0.5
0.0    0.5
dtype: float64

In [81]:
pd.Series(y_train).value_counts(normalize = True)

0.0    0.517647
1.0    0.482353
dtype: float64

In [82]:
len(X_train)

85

In [83]:
len(X_val)

10

In [84]:
len(X_test)

24

1. Hidden state initialize differently every time running LSTM??

2. init_hidden?

3. Softmax?

4. Use only hidden state, not cell state or output?

5. Results somewhat random?

6. Hidden Dimension?

Fine-tune embedding

Same initial hidden states DONE

Check dropout in evaluation DONE no dropout

Validation set DONE

Check fully connected weights 

Add f.c. layers DOING

Mean + std of several runs



In [52]:
def run_lstm_one_sentence(sentence, title, lstm, hidden, embedding_index):
    inputs = point2mn(sentence, title, embedding_index)
    tensor_input = [torch.tensor([x]) for x in inputs]
    tensor_input = torch.cat(tensor_input).view(1, len(tensor_input), -1)
#     hidden = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM)) #???????
    tensor_output, hidden = lstm(tensor_input, hidden)
    # should return hidden state and cell state in 'hidden' instead of output
    return hidden

In [59]:

def run_lstm_on_fid(fid, embedding_index, lstm, hidden, combine_func = [torch.mean, torch.max]):
    """
    Given fid, find main points for the debate and for each sentence, pass corresponding 
    matching vectors to lstm and get hidden state in the end. Gather the hidden states
    in a list and do combine_func.
    -combine_func: the funtion applied to combine lstm outputs from one side elementwisely
    """
    
    title = main_points[main_points['id'] == fid].title.iloc[0]
    for_main_points, against_main_points = get_main_points(fid, main_points)
    for_output_list = []
    against_output_list = []
    
    for sentence in for_main_points:
        hidden_state, cell_state = run_lstm_one_sentence(sentence, title, lstm, hidden, embedding_index)
        for_output_list.append(hidden_state)
        
    for sentence in against_main_points:
        hidden_state, cell_state = run_lstm_one_sentence(sentence, title, lstm, hidden, embedding_index)
        against_output_list.append(hidden_state)
    for_torchs = []
    against_torchs = []
    for combine_f in combine_func:
        if combine_f == torch.mean:
            for_torch = combine_f(torch.stack(for_output_list), dim = 0, keepdim = True)#[0]
            against_torch = combine_f(torch.stack(against_output_list), dim = 0, keepdim = True)#[0]
        else:
            for_torch = combine_f(torch.stack(for_output_list), dim = 0, keepdim = True)[0]
            against_torch = combine_f(torch.stack(against_output_list), dim = 0, keepdim = True)[0]
        for_torchs.append(for_torch)
        against_torchs.append(against_torch)
    if len(for_torchs) == 2:
        t1 = torch.cat((for_torchs[0], for_torchs[1]), dim = 2)
        t2 = torch.cat((against_torchs[0], against_torchs[1]), dim = 2)
        return torch.cat((t1, t2), dim = 2)
    return torch.cat((for_torchs[0], against_torchs[0]), dim = 2)

In [90]:
class LSTMNet(nn.Module):
    def __init__(self, output_size_1, output_size_2, n_layers, embedding_dim, hidden_dim, combine_funcs, hidden, drop_prob=0.5):
        super(LSTMNet, self).__init__()
        self.output_size_1 = output_size_1
        self.output_size_2 = output_size_2
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.combine_funcs = combine_funcs
        self.hidden = hidden

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(hidden_dim * 2 * len(self.combine_funcs), self.output_size_1)
        self.fc2 = nn.Linear(output_size_1, self.output_size_2)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, fids):
        batch_size = len(fids)
        lstm_out = torch.stack([run_lstm_on_fid(fid, embedding_index, self.lstm, self.hidden, \
                                                combine_func = self.combine_funcs) for fid in fids])
    
        lstm_out = lstm_out.contiguous().view(-1, len(self.combine_funcs) * 2 * self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out#, hidden
    
#     def init_hidden(self, batch_size):
#         weight = next(self.parameters()).data
#         hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
#                       weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
#         return hidden

In [147]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 50
OUTPUT_SIZE_1 = 20
COMBINE_FUNCS =[torch.mean, torch.max]
OUTPUT_SIZE_2 = 1
N_LAYERS = 1
DROPOUT_PROB = 0.5
HIDDEN_INITIAL = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM))
# all should use the same one lstm layer
# lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM)

In [148]:
model = LSTMNet(OUTPUT_SIZE_1, OUTPUT_SIZE_2, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, COMBINE_FUNCS, HIDDEN_INITIAL,DROPOUT_PROB)

In [149]:
lr = 0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [150]:
epochs = 20
counter = 0
# print_every = 1000
# clip = 5
# valid_loss_min = np.Inf
train_acc = []
train_acc2 = []

val_acc = []
test_acc = []
model.train()
for i in range(epochs):
#     h = model.init_hidden(batch_size)
    
#     for inputs, labels in train_loader:
#         counter += 1
#         h = tuple([e.data for e in h])
#         inputs, labels = inputs.to(device), labels.to(device)
    model.train()
    model.zero_grad()
    output = model(X_train)
    loss = criterion(output.squeeze(), y_train.float())
    print(loss.item())
    loss.backward()
#     nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    model.eval()
    outs = model(X_train)
    train_acc.append(torch.sum((outs > 0.5) == y_train).item() / len(y_train))

    outs = model(X_val)
    val_acc.append(torch.sum((outs > 0.5) == y_val).item() / len(y_val))
    
    outs = model(X_test)
    test_acc.append(torch.sum((outs > 0.5) == y_test).item() / len(y_test))

    
    
#         if counter%print_every == 0:
#             val_h = model.init_hidden(batch_size)
#             val_losses = []
#             model.eval()
#             for inp, lab in val_loader:
#                 val_h = tuple([each.data for each in val_h])
#                 inp, lab = inp.to(device), lab.to(device)
#                 out, val_h = model(inp, val_h)
#                 val_loss = criterion(out.squeeze(), lab.float())
#                 val_losses.append(val_loss.item())
                
#             model.train()
#             print("Epoch: {}/{}...".format(i+1, epochs),
#                   "Step: {}...".format(counter),
#                   "Loss: {:.6f}...".format(loss.item()),
#                   "Val Loss: {:.6f}".format(np.mean(val_losses)))
#             if np.mean(val_losses) <= valid_loss_min:
#                 torch.save(model.state_dict(), './state_dict.pt')
#                 print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
#                 valid_loss_min = np.mean(val_losses)

0.6946223974227905
0.6832095980644226
0.6761366128921509
0.6420382857322693
0.6203120350837708
0.5813390016555786
0.5190531611442566
0.4666883945465088
0.4329531192779541
0.3752633035182953
0.30831459164619446
0.23986196517944336
0.24928957223892212
0.39366015791893005
0.1403120756149292
0.2522309720516205
0.12413179874420166
0.1029122844338417
0.16996270418167114
0.08095245808362961


In [151]:
def training_results(outputsize, N_LAYERS, embedding_dim, hidden_dim, combine_funcs, drop_prob, lr, epochs):
    print('output size = ', outputsize, 'n_layers =', N_LAYERS, 'embedding_dim = ', embedding_dim, \
         'hidden_dim = ', hidden_dim, 'combine_funcs = ', str(combine_funcs), 'drop_prob = ', drop_prob, \
          'lr = ', lr, 'epochs = ', epochs)
    return pd.DataFrame({'# Epoch': range(1,epochs + 1), 'training acc': train_acc,\
                         'validation acc': val_acc, 'testing acc': test_acc})

In [152]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], DROPOUT_PROB, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  50 combine_funcs =  [<built-in method mean of type object at 0x116f7deb0>, <built-in method max of type object at 0x116f7deb0>] drop_prob =  0.5 lr =  0.005 epochs =  20


,# Epoch,training acc,validation acc,testing acc
0,1,0.647059,0.4,0.583333
1,2,0.788235,0.5,0.666667
2,3,0.623529,0.5,0.500000
3,4,0.658824,0.6,0.500000
4,5,0.894118,0.7,0.583333
5,6,0.870588,0.6,0.500000
6,7,0.776471,0.6,0.583333
7,8,0.811765,0.5,0.458333
8,9,0.800000,0.7,0.541667
9,10,0.858824,0.6,0.458333


In [141]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], DROPOUT_PROB, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  50 combine_funcs =  [<built-in method mean of type object at 0x116f7deb0>, <built-in method max of type object at 0x116f7deb0>] drop_prob =  0.5 lr =  0.005 epochs =  20


,# Epoch,training acc,validation acc,testing acc
0,1,0.658824,0.6,0.541667
1,2,0.705882,0.6,0.583333
2,3,0.835294,0.8,0.625000
3,4,0.729412,0.5,0.500000
4,5,0.788235,0.5,0.625000
5,6,0.870588,0.6,0.625000
6,7,0.882353,0.8,0.625000
7,8,0.870588,0.9,0.625000
8,9,0.811765,0.6,0.541667
9,10,0.764706,0.8,0.541667


In [119]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], DROPOUT_PROB, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  30 combine_funcs =  [<built-in method mean of type object at 0x116f7deb0>, <built-in method max of type object at 0x116f7deb0>] drop_prob =  0.6 lr =  0.01 epochs =  20


,# Epoch,training acc,validation acc,testing acc
0,1,0.529412,0.6,0.500000
1,2,0.764706,0.8,0.666667
2,3,0.835294,0.7,0.541667
3,4,0.823529,0.7,0.583333
4,5,0.823529,0.6,0.625000
5,6,0.835294,0.7,0.583333
6,7,0.870588,0.7,0.666667
7,8,0.823529,0.7,0.541667
8,9,0.776471,0.5,0.583333
9,10,0.882353,0.7,0.541667


In [106]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], DROPOUT_PROB, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  50 combine_funcs =  [<built-in method mean of type object at 0x116f7deb0>, <built-in method max of type object at 0x116f7deb0>] drop_prob =  0.6 lr =  0.01 epochs =  20


,# Epoch,training acc,validation acc,testing acc
0,1,0.717647,0.7,0.625000
1,2,0.541176,0.5,0.458333
2,3,0.564706,0.5,0.541667
3,4,0.682353,0.5,0.500000
4,5,0.917647,0.6,0.666667
5,6,0.800000,0.7,0.541667
6,7,0.647059,0.5,0.500000
7,8,0.905882,0.6,0.625000
8,9,0.705882,0.6,0.541667
9,10,0.964706,0.6,0.583333


In [101]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], DROPOUT_PROB, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  50 combine_funcs =  [<built-in method mean of type object at 0x116f7deb0>, <built-in method max of type object at 0x116f7deb0>] drop_prob =  0.6 lr =  0.01 epochs =  20


,# Epoch,training acc,validation acc,testing acc
0,1,0.517647,0.5,0.500000
1,2,0.494118,0.5,0.500000
2,3,0.670588,0.5,0.541667
3,4,0.588235,0.5,0.500000
4,5,0.647059,0.5,0.500000
5,6,0.882353,0.6,0.583333
6,7,0.858824,0.6,0.583333
7,8,0.847059,0.7,0.583333
8,9,0.776471,0.8,0.541667
9,10,0.776471,0.6,0.500000


In [149]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], DROPOUT_PROB, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  100 combine_funcs =  [<built-in method mean of type object at 0x115fcceb0>, <built-in method max of type object at 0x115fcceb0>] drop_prob =  0.7 lr =  0.001 epochs =  20


,# Epoch,training acc,testing acc
0,1,0.547368,0.500000
1,2,0.568421,0.500000
2,3,0.610526,0.500000
3,4,0.631579,0.500000
4,5,0.694737,0.500000
5,6,0.736842,0.500000
6,7,0.789474,0.541667
7,8,0.800000,0.416667
8,9,0.831579,0.458333
9,10,0.863158,0.541667


In [143]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], 0.4, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  100 combine_funcs =  [<built-in method mean of type object at 0x115fcceb0>, <built-in method max of type object at 0x115fcceb0>] drop_prob =  0.4 lr =  0.001 epochs =  20


,# Epoch,training acc,testing acc
0,1,0.536842,0.500000
1,2,0.631579,0.458333
2,3,0.705263,0.416667
3,4,0.726316,0.458333
4,5,0.778947,0.458333
5,6,0.789474,0.458333
6,7,0.789474,0.458333
7,8,0.789474,0.500000
8,9,0.789474,0.500000
9,10,0.768421,0.458333


In [138]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], 0.4, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  100 combine_funcs =  [<built-in method mean of type object at 0x115fcceb0>, <built-in method max of type object at 0x115fcceb0>] drop_prob =  0.4 lr =  0.001 epochs =  20


,# Epoch,training acc,testing acc
0,1,0.547368,0.625000
1,2,0.652632,0.541667
2,3,0.673684,0.583333
3,4,0.747368,0.583333
4,5,0.757895,0.583333
5,6,0.778947,0.625000
6,7,0.789474,0.666667
7,8,0.821053,0.666667
8,9,0.873684,0.666667
9,10,0.884211,0.666667


In [133]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], 0.4, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  50 combine_funcs =  [<built-in method mean of type object at 0x115fcceb0>, <built-in method max of type object at 0x115fcceb0>] drop_prob =  0.4 lr =  0.001 epochs =  20


,# Epoch,training acc,testing acc
0,1,0.505263,0.500000
1,2,0.568421,0.500000
2,3,0.631579,0.458333
3,4,0.747368,0.541667
4,5,0.810526,0.500000
5,6,0.800000,0.500000
6,7,0.831579,0.458333
7,8,0.821053,0.500000
8,9,0.831579,0.541667
9,10,0.810526,0.541667


In [125]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], 0.4, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  50 combine_funcs =  [<built-in method mean of type object at 0x115fcceb0>, <built-in method max of type object at 0x115fcceb0>] drop_prob =  0.4 lr =  0.001 epochs =  20


,# Epoch,training acc,testing acc
0,1,0.547368,0.583333
1,2,0.568421,0.500000
2,3,0.652632,0.541667
3,4,0.663158,0.625000
4,5,0.747368,0.666667
5,6,0.757895,0.708333
6,7,0.800000,0.708333
7,8,0.821053,0.708333
8,9,0.821053,0.708333
9,10,0.842105,0.708333


In [120]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], 0.4, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  50 combine_funcs =  [<built-in method mean of type object at 0x115fcceb0>, <built-in method max of type object at 0x115fcceb0>] drop_prob =  0.4 lr =  0.001 epochs =  20


,# Epoch,training acc,testing acc
0,1,0.873684,0.583333
1,2,0.873684,0.625000
2,3,0.873684,0.625000
3,4,0.884211,0.541667
4,5,0.884211,0.541667
5,6,0.894737,0.541667
6,7,0.905263,0.583333
7,8,0.915789,0.541667
8,9,0.905263,0.583333
9,10,0.905263,0.625000


In [161]:
training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], DROPOUT_PROB, lr, epochs)

output size =  1 n_layers = 1 embedding_dim =  200 hidden_dim =  50 combine_funcs =  [<built-in method mean of type object at 0x115fcceb0>, <built-in method max of type object at 0x115fcceb0>] drop_prob =  0.7 lr =  0.01 epochs =  20


,# Epoch,training acc,testing acc
0,1,0.515789,0.500000
1,2,0.726316,0.583333
2,3,0.778947,0.583333
3,4,0.747368,0.458333
4,5,0.757895,0.583333
5,6,0.778947,0.500000
6,7,0.800000,0.541667
7,8,0.684211,0.541667
8,9,0.757895,0.583333
9,10,0.884211,0.541667


In [167]:
# training_results(OUTPUT_SIZE, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, [torch.mean, torch.max], DROPOUT_PROB, lr, epochs)

In [104]:
epochs = 10
counter = 0
print_every = 1000
# clip = 5
valid_loss_min = np.Inf
train_acc = []
test_acc = []
model.train()
for i in range(epochs):
#     h = model.init_hidden(batch_size)
    
#     for inputs, labels in train_loader:
#         counter += 1
#         h = tuple([e.data for e in h])
#         inputs, labels = inputs.to(device), labels.to(device)
    model.zero_grad()
    output = model(X_train)
    loss = criterion(output.squeeze(), y_train.float())
    print(loss)
    loss.backward()
#     nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    outs = model(X_train)
    train_acc.append(torch.sum((outs > 0.5) == y_train).item() / len(y_train))
    outs = model(X_test)
    test_acc.append(torch.sum((outs > 0.5) == y_test).item() / len(y_test))
    
    
#         if counter%print_every == 0:
#             val_h = model.init_hidden(batch_size)
#             val_losses = []
#             model.eval()
#             for inp, lab in val_loader:
#                 val_h = tuple([each.data for each in val_h])
#                 inp, lab = inp.to(device), lab.to(device)
#                 out, val_h = model(inp, val_h)
#                 val_loss = criterion(out.squeeze(), lab.float())
#                 val_losses.append(val_loss.item())
                
#             model.train()
#             print("Epoch: {}/{}...".format(i+1, epochs),
#                   "Step: {}...".format(counter),
#                   "Loss: {:.6f}...".format(loss.item()),
#                   "Val Loss: {:.6f}".format(np.mean(val_losses)))
#             if np.mean(val_losses) <= valid_loss_min:
#                 torch.save(model.state_dict(), './state_dict.pt')
#                 print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
#                 valid_loss_min = np.mean(val_losses)

torch.Size([95, 800])
tensor(0.6929, grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([95, 800])
torch.Size([24, 800])
torch.Size([95, 800])
tensor(0.6958, grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([95, 800])
torch.Size([24, 800])
torch.Size([95, 800])
tensor(0.6592, grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([95, 800])
torch.Size([24, 800])
torch.Size([95, 800])
tensor(0.6423, grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([95, 800])
torch.Size([24, 800])
torch.Size([95, 800])
tensor(0.6292, grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([95, 800])
torch.Size([24, 800])
torch.Size([95, 800])
tensor(0.5868, grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([95, 800])
torch.Size([24, 800])
torch.Size([95, 800])
tensor(0.5452, grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([95, 800])
torch.Size([24, 800])
torch.Size([95, 800])
tensor(0.4975, grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([95, 800])
torch.Size([24, 800])
torch.Size([95, 800])
tensor(0.6022, grad_fn=<Bi

In [105]:
train_acc

[0.5157894736842106,
 0.5263157894736842,
 0.6210526315789474,
 0.7052631578947368,
 0.7052631578947368,
 0.8842105263157894,
 0.7578947368421053,
 0.6,
 0.8105263157894737,
 0.7263157894736842]

In [106]:
test_acc

[0.5,
 0.5416666666666666,
 0.5833333333333334,
 0.5,
 0.625,
 0.6666666666666666,
 0.5,
 0.5833333333333334,
 0.5833333333333334,
 0.5]

In [35]:
# outs = model(X_train)
# torch.sum((outs > 0.5) == y_train).item() / len(y_train)

In [200]:
outs

tensor([0.5020, 0.3077, 0.3503, 0.5805, 0.4688, 0.5605, 0.1735, 0.3698, 0.3980,
        0.5669, 0.5357, 0.4020, 0.3885, 0.4056, 0.3149, 0.6157, 0.5548, 0.5643,
        0.3511, 0.5886, 0.3619, 0.5594, 0.4987, 0.6190],
       grad_fn=<SelectBackward>)

In [19]:
# class FullyNet(nn.Module):
#     def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
#         super(FullyNet, self).__init__()
#         self.output_size = output_size
#         self.n_layers = n_layers
#         self.hidden_dim = hidden_dim
        
#         #self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
#         self.dropout = nn.Dropout(drop_prob)
#         self.fc = nn.Linear(hidden_dim, output_size)
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self, x, hidden):
#         batch_size = x.size(0)
#         x = x.long()
#         #embeds = self.embedding(x)
#         lstm_out, hidden = self.lstm(embeds, hidden)
#         lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
#         out = self.dropout(lstm_out)
#         out = self.fc(out)
#         out = self.sigmoid(out)
        
#         out = out.view(batch_size, -1)
#         out = out[:,-1]
#         return out, hidden